In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import tensorflow as tf
import itertools as it
import pickle
import os

from tensorflow.contrib.tensorboard.plugins import projector

from time import time

from importlib import reload
import sys
if '..' not in sys.path:
    sys.path.append('..')
from sertf import core
reload(core)

LOG_DIR = '/tmp/tensorboard-logs/semantic/'

PATH_DATA = '../data/amazon/food/reviews_df.msg'
PATH_ENC_TXT = '../data/amazon/food/reviews_txt_enc_s.msg'
PATH_VOCAB = '../data/amazon/food/vocab.p'

entity_col = 'ProductId'

In [2]:
df = pd.read_msgpack(PATH_DATA)
data_words_enc = pd.read_msgpack('../data/amazon/food/reviews_txt_enc_s.msg')
vocab = pickle.load(open(PATH_VOCAB, 'rb'))

In [3]:
n_entities = len(df[entity_col].cat.categories)

In [4]:
entity_codes = df[entity_col].cat.codes.values

In [5]:
model = core.Model(vocab, n_entities)

In [6]:
gen = core.win_gen(data_words_enc, entity_codes, n_entities,
                        model.n_negs_per_pos,
                        model.ph_d, model.batch_size)

In [8]:
%%time
max_steps = 100

print(f'Approx # epochs: {max_steps*model.batch_size/len(df)}')

gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
saver = tf.train.Saver()
summary_writer = tf.summary.FileWriter(LOG_DIR)

with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    sess.run(tf.global_variables_initializer())
    tic = time()
    for step in range(max_steps):
        feed = next(gen)
        sess.run(model.train_op, feed_dict=feed)
    

Approx # epochs: 0.18013770683291877
CPU times: user 2.3 s, sys: 232 ms, total: 2.53 s
Wall time: 1.9 s
